In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(""), "..")))


import json

import src.constants_logistics as cs

import pandas as pd

from datetime import date, datetime, timedelta

import src.helpers as hlp

import src.train.train as train

# Load data

In [ ]:


f = open(cs.PATH_TO_RAW_DATA,)

data = json.load(f)

df = pd.DataFrame.from_dict(data)

new_code = 2*5


# Get data for product

In [ ]:
def get_data_for_specific_product(product_name,last_used_data, df):
    
    df = df.loc[df["product_name"] == product_name].reset_index()

    df["requested_delivery_date"] = pd.to_datetime(df["requested_delivery_date"], format="%Y-%m-%d")

    df = df.loc[df["requested_delivery_date"] < last_used_date]

    missing_dates =  get_missing_dates(df.loc[:, "requested_delivery_date"])

    time_series_df = df.copy()

    for date in missing_dates:
        row = [
            {
                "product_name": time_series_df.loc[0, "product_name"],
                "product_description": time_series_df.loc[0, "product_description"],
                "sales_quantity": 0.0,
                "date_of_sale": "",
                "requested_delivery_date": date
            }
        ]
    time_series_df = time_series_df.append(row, ignore_index=True)

    time_series_df = time_series_df.sort_values(by="requested_delivery_date", ignore_index=True)

    time_series_df.index = time_series_df["requested_delivery_date"]

    time_series_df = time_series_df.resample("M", on="requested_delivery_date").sum().reset_index().sort_values(by="requested_delivery_date")

    return time_series_df

# 
def get_missing_dates(dates_column):
    dates = pd.to_datetime(dates_column, format="%Y-%m-%d")
    d = [date for date in dates]
    min_date = min(d)
    max_date = max(d)
    date_set = set(min_date + timedelta(x) for x in range((max_date - min_date).days))
    return [date for date in sorted(date_set - set(d))]


# Create datasets

In [ ]:

product_name = "146092"

product_description = df[df["product_name"] == product_name].reset_index().loc[0, "product_description"]

last_used_date = datetime(2020,2,1) 

time_series_df = get_data_for_specific_product(product_name= product_name, last_used_data=last_used_date, df=df)

x = time_series_df["requested_delivery_date"].values
y = time_series_df["sales_quantity"].values
data_title = f"{product_name} {product_description}"
horizon = 6


In [ ]:
# Set up the train pipeline, tune models and evaluate them
train_pipeline = train.TrainPipeline(
    x=x, y=y, data_title=data_title, horizon=horizon
)

train_pipeline.tune_model_params()
train_pipeline.evaluate_models(show_plot=True, show_avg_rmse=True)